In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import zipfile
from io import BytesIO
import warnings
from ipywidgets import FloatProgress, HTML, VBox, IntSlider, Button, HBox, Label, Output
from ipywidgets import Dropdown, DatePicker
from IPython.display import display, clear_output

# Widget pour sélectionner le symbole
symbol_dropdown = Dropdown(
    options=['BTCUSDT', 'ETHUSDT'],
    value='BTCUSDT',
    description='Symbole:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)
duree_slider = IntSlider(
    value=365,
    min=1,
    max=730,
    step=1,
    description='Durée (jours):',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner l'intervalle
from ipywidgets import Dropdown
interval_dropdown = Dropdown(
    options=['1m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1M'],
    value='1h',
    description='Intervalle:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la date de fin
end_date_picker = DatePicker(
    description='Date de fin:',
    value=datetime.now().date(),
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la période de la moyenne mobile simple (SMA)
sma_period_slider = IntSlider(
    value=10,
    min=2,
    max=200,
    step=1,
    description='SMA Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la période de la moyenne mobile exponentielle (EMA)
ema_period_slider = IntSlider(
    value=50,
    min=2,
    max=200,
    step=1,
    description='EMA Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la période de la deuxième moyenne mobile exponentielle (EMA2)
ema2_period_slider = IntSlider(
    value=200,
    min=2,
    max=200,
    step=1,
    description='EMA 2 Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)


# Widget pour sélectionner le nombre de pics FFT
top_freq_slider = IntSlider(
    value=200,
    min=1,
    max=500,
    step=1,
    description='FFT Pics:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la période RSI
rsi_period_slider = IntSlider(
    value=14,
    min=2,
    max=50,
    step=1,
    description='RSI Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour sélectionner la période de la Hull Moving Average (HMA)
hma_period_slider = IntSlider(
    value=20,
    min=2,
    max=200,
    step=1,
    description='HMA Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour ZLEMA
zlema_period_slider = IntSlider(
    value=14,
    min=2,
    max=200,
    step=1,
    description='ZLEMA Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Widget pour TEMA
tema_period_slider = IntSlider(
    value=14,
    min=2,
    max=200,
    step=1,
    description='TEMA Period:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)


# Checkboxes pour contrôler l'affichage des éléments du graphique
from ipywidgets import Checkbox
show_price_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_sma_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_ema_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_ema2_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_hma_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_zlema_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_tema_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_volume_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_diff_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_rsi_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_wavelet_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_fft_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})
show_reconstruction_checkbox = Checkbox(value=True, description='', style={'description_width': '0px'}, layout={'width': 'auto'})

# Widget pour sélectionner le nombre de pics FFT

# Bouton pour lancer le téléchargement
download_button = Button(
    description='Télécharger',
    button_style='primary',
    tooltip='Cliquez pour lancer le téléchargement des données Binance Vision'
)

# Bouton pour lancer la visualisation
visualize_button = Button(
    description='Visualiser',
    button_style='success',
    tooltip='Cliquez pour afficher les graphiques d\'analyse (nécessite des données téléchargées)',
    disabled=True  # Désactivé par défaut
)

# Zone de sortie pour les messages
output_area = Output()

# Variables globales (seront mises à jour par les widgets)
DUREE_JOURS = duree_slider.value
INTERVAL = interval_dropdown.value
SYMBOL = symbol_dropdown.value
END_DATE = end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else datetime.now().strftime('%Y-%m-%d')
SMA_PERIOD = sma_period_slider.value
EMA_PERIOD = ema_period_slider.value
EMA2_PERIOD = ema2_period_slider.value
HMA_PERIOD = hma_period_slider.value
ZLEMA_PERIOD = zlema_period_slider.value
TEMA_PERIOD = tema_period_slider.value
TOP_FREQUENCIES = top_freq_slider.value
RSI_PERIOD = rsi_period_slider.value

# Variables globales pour les checkboxes d'affichage
SHOW_PRICE = show_price_checkbox.value
SHOW_SMA = show_sma_checkbox.value
SHOW_EMA = show_ema_checkbox.value
SHOW_EMA2 = show_ema2_checkbox.value
SHOW_HMA = show_hma_checkbox.value
SHOW_ZLEMA = show_zlema_checkbox.value
SHOW_TEMA = show_tema_checkbox.value
SHOW_VOLUME = show_volume_checkbox.value
SHOW_DIFF = show_diff_checkbox.value
SHOW_RSI = show_rsi_checkbox.value
SHOW_WAVELET = show_wavelet_checkbox.value
SHOW_FFT = show_fft_checkbox.value
SHOW_RECONSTRUCTION = show_reconstruction_checkbox.value

# Initialiser les variables globales pour éviter les erreurs
btc_data = []
df_btc = pd.DataFrame()

# Fonction callback pour le bouton de téléchargement
def on_download_click(b):
    global DUREE_JOURS, INTERVAL, btc_data, df_btc
    
    # Désactiver le bouton et changer son texte pendant le téléchargement
    original_button_text = download_button.description
    download_button.description = '⏳ Téléchargement...'
    download_button.disabled = True
    
    try:
        with output_area:
            clear_output(wait=True)
            print("📥 Téléchargement des données en cours...")
            print(f"Symbole: {symbol_dropdown.value}, Durée: {duree_slider.value} jours")
            print()
            
            # Mettre à jour les variables globales
            DUREE_JOURS = duree_slider.value
            INTERVAL = interval_dropdown.value
            SYMBOL = symbol_dropdown.value
            END_DATE = end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else datetime.now().strftime('%Y-%m-%d')
            
            # Lancer le téléchargement
            btc_data = download_binance_vision_data(SYMBOL, INTERVAL, DUREE_JOURS)
            
            # Créer le DataFrame
            df_btc = create_dataframe(btc_data, SYMBOL)
            
            if len(df_btc) > 0:
                print("✅ Téléchargement terminé avec succès!")
                print(f"Données récupérées: {len(df_btc)} périodes")
                print(f"Période: {df_btc.index.min()} à {df_btc.index.max()}")
                # Activer le bouton de visualisation
                visualize_button.disabled = False
                print("\n🔄 Génération automatique de la visualisation...")
                print()
                
                # Lancer automatiquement la visualisation
                clear_output(wait=True)
                create_visualization()
            else:
                print("\n❌ Échec du téléchargement - Aucune donnée récupérée")
                visualize_button.disabled = True
    finally:
        # Réactiver le bouton et remettre le texte original
        download_button.description = original_button_text
        download_button.disabled = False

# Fonction callback pour le bouton de visualisation
def on_visualize_click(b):
    global df_btc
    
    # Désactiver le bouton et changer son texte pendant le calcul
    original_button_text = visualize_button.description
    visualize_button.description = '⏳ Calcul en cours...'
    visualize_button.disabled = True
    
    try:
        with output_area:
            clear_output(wait=True)
            print("🔄 Génération des visualisations en cours...")
            print(f"Analyse de {len(df_btc)} périodes de {symbol_dropdown.value}")
            print()
            
            # Exécuter la visualisation
            create_visualization()
            
            print("✅ Visualisation terminée !")
    finally:
        # Réactiver le bouton et remettre le texte original
        visualize_button.description = original_button_text
        visualize_button.disabled = False

# Fonction pour créer les visualisations
def create_visualization():
    global SMA_PERIOD, EMA_PERIOD, EMA2_PERIOD, HMA_PERIOD, ZLEMA_PERIOD, TEMA_PERIOD, TOP_FREQUENCIES, RSI_PERIOD  # Accéder aux variables globales
    global SHOW_PRICE, SHOW_SMA, SHOW_EMA, SHOW_EMA2, SHOW_HMA, SHOW_ZLEMA, SHOW_TEMA, SHOW_VOLUME, SHOW_DIFF, SHOW_RSI, SHOW_WAVELET, SHOW_FFT, SHOW_RECONSTRUCTION  # Variables des checkboxes
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np
    import pywt  # PyWavelets pour l'analyse par ondelettes

    # Paramètres configurables
    # SMA_PERIOD, EMA_PERIOD et TOP_FREQUENCIES sont maintenant définis globalement par les widgets

    if len(df_btc) > 0:
        print("📊 Calcul des moyennes mobiles...")
        # Calculer la moyenne mobile simple (SMA) sur les SMA_PERIOD dernières périodes
        df_btc[f'sma_{SMA_PERIOD}'] = df_btc['close'].rolling(window=SMA_PERIOD).mean()
        # Décalage vers la gauche pour aligner (la fin des données n'est pas importante)
        #decalage_sma = SMA_PERIOD // 2
        # df_btc[f'sma_{SMA_PERIOD}'] = df_btc[f'sma_{SMA_PERIOD}'].shift(-decalage_sma)

        # Calculer la moyenne mobile exponentielle (EMA)
        df_btc[f'ema_{EMA_PERIOD}'] = df_btc['close'].ewm(span=EMA_PERIOD, adjust=False).mean()
        # Décalage vers la gauche pour aligner (la fin des données n'est pas importante)
        #decalage_ema = EMA_PERIOD // 2
        # df_btc[f'ema_{EMA_PERIOD}'] = df_btc[f'ema_{EMA_PERIOD}'].shift(-decalage_ema)

        # Calculer la deuxième moyenne mobile exponentielle (EMA2)
        df_btc[f'ema2_{EMA2_PERIOD}'] = df_btc['close'].ewm(span=EMA2_PERIOD, adjust=False).mean()
        #decalage_ema2 = EMA2_PERIOD // 2
        #df_btc[f'ema2_{EMA2_PERIOD}'] = df_btc[f'ema2_{EMA2_PERIOD}'].shift(-decalage_ema2)

        # Calculer la Hull Moving Average (HMA)
        def wma(series, period):
            return series.rolling(window=period).apply(lambda x: np.sum(np.arange(1, period + 1) * x) / (period * (period + 1) / 2), raw=True)

        period = HMA_PERIOD
        half_period = int(period / 2)
        sqrt_period = int(np.sqrt(period))
        
        wma_half = wma(df_btc['close'], half_period)
        wma_full = wma(df_btc['close'], period)
        
        df_btc[f'hma_{HMA_PERIOD}'] = wma(2 * wma_half - wma_full, sqrt_period)

        # Calculer la Zero Lag Exponential Moving Average (ZLEMA)
        lag = (ZLEMA_PERIOD - 1) // 2
        ema_data = df_btc['close'] + (df_btc['close'].diff(lag))
        df_btc[f'zlema_{ZLEMA_PERIOD}'] = ema_data.ewm(span=ZLEMA_PERIOD, adjust=False).mean()

        # Calculer la Triple Exponential Moving Average (TEMA)
        ema1 = df_btc['close'].ewm(span=TEMA_PERIOD, adjust=False).mean()
        ema2 = ema1.ewm(span=TEMA_PERIOD, adjust=False).mean()
        ema3 = ema2.ewm(span=TEMA_PERIOD, adjust=False).mean()
        df_btc[f'tema_{TEMA_PERIOD}'] = (3 * ema1) - (3 * ema2) + ema3


        # Calculer la différence entre close et la SMA décalée
        df_btc['diff_close_sma'] = df_btc['close'] - df_btc[f'sma_{SMA_PERIOD}']

        # Calculer le RSI (Relative Strength Index)
        def calculate_rsi(data, period=14):
            delta = data.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        
        df_btc['rsi'] = calculate_rsi(df_btc['close'], period=RSI_PERIOD)

        # Série de différences (sans NaN)
        diff_series = df_btc['diff_close_sma'].dropna()
        difference_values = diff_series.values

        # === DÉTERMINER LES ÉLÉMENTS À AFFICHER ===
        plot_elements = []
        
        # Prix/Volume (toujours ensemble si au moins un des deux est activé)
        if SHOW_PRICE or SHOW_SMA or SHOW_EMA or SHOW_EMA2 or SHOW_HMA or SHOW_ZLEMA or SHOW_TEMA or SHOW_VOLUME:
            plot_elements.append(('price_volume', 'Prix, Moyennes Mobiles & Volume'))
        
        # Logique de regroupement pour RSI, Diff et Reconstruction
        show_combined_rsi_diff = SHOW_RSI and (SHOW_DIFF or SHOW_RECONSTRUCTION)
        show_rsi_only = SHOW_RSI and not (SHOW_DIFF or SHOW_RECONSTRUCTION)
        show_diff_recon_only = (SHOW_DIFF or SHOW_RECONSTRUCTION) and not SHOW_RSI

        if show_combined_rsi_diff:
            plot_elements.append(('rsi_diff_recon', 'RSI & Différence + Reconstruction'))
        elif show_rsi_only:
            plot_elements.append(('rsi_only', 'RSI'))
        elif show_diff_recon_only:
            plot_elements.append(('diff_recon_only', 'Différence + Reconstruction'))

        if SHOW_WAVELET:
            plot_elements.append(('wavelet', 'Ondelettes sur Différence'))
        
        if SHOW_FFT:
            plot_elements.append(('fft', 'Spectre FFT (Périodes)'))

        # Si aucun élément n'est sélectionné, afficher un message
        if not plot_elements:
            print("⚠ Aucun élément d'affichage sélectionné")
            return

        # === CRÉATION DYNAMIQUE DES SUBPLOTS ===
        num_plots = len(plot_elements)
        subplot_specs = []
        subplot_titles = []
        
        for element, title in plot_elements:
            if element in ['price_volume', 'rsi_diff_recon']:
                # Ces graphiques ont un axe Y secondaire
                subplot_specs.append([{"secondary_y": True}])
            else:
                # Les autres n'en ont pas
                subplot_specs.append([{"secondary_y": False}])
            subplot_titles.append(title)

        fig = make_subplots(
            rows=num_plots,
            cols=1,
            shared_xaxes=False,  # On va gérer manuellement la synchronisation
            vertical_spacing=0.10,  # Augmenté pour éviter le chevauchement des axes X
            specs=subplot_specs,
            subplot_titles=subplot_titles
        )

        # === AJOUT DES TRACES SELON LES ÉLÉMENTS ACTIVÉS ===
        current_row = 1
        
        # --- Rangée 1: Prix, MMs, Volume ---
        if any(e for e, t in plot_elements if e == 'price_volume'):
            # Prix + MMs sur l'axe Y primaire
            if SHOW_PRICE:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['close'], mode='lines', name=f"{symbol_dropdown.value} Close", line=dict(color='red', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_SMA:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'sma_{SMA_PERIOD}'], mode='lines', name=f"SMA {SMA_PERIOD}", line=dict(color='blue', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_EMA:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'ema_{EMA_PERIOD}'], mode='lines', name=f"EMA {EMA_PERIOD}", line=dict(color='cyan', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_EMA2:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'ema2_{EMA2_PERIOD}'], mode='lines', name=f"EMA {EMA2_PERIOD}", line=dict(color='magenta', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_HMA:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'hma_{HMA_PERIOD}'], mode='lines', name=f"HMA {HMA_PERIOD}", line=dict(color='yellow', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_ZLEMA:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'zlema_{ZLEMA_PERIOD}'], mode='lines', name=f"ZLEMA {ZLEMA_PERIOD}", line=dict(color='orange', width=1)), row=current_row, col=1, secondary_y=False)
            if SHOW_TEMA:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc[f'tema_{TEMA_PERIOD}'], mode='lines', name=f"TEMA {TEMA_PERIOD}", line=dict(color='purple', width=1)), row=current_row, col=1, secondary_y=False)

            # Volume sur l'axe Y secondaire
            if SHOW_VOLUME:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['volume'], mode='lines', name='Volume', line=dict(color='green', width=1), opacity=0.3), row=current_row, col=1, secondary_y=True)
            
            current_row += 1

        # --- Rangée 2: RSI, Diff, Reconstruction (logique conditionnelle) ---
        
        # Cas 1: RSI combiné avec Diff/Reconstruction
        if any(e for e, t in plot_elements if e == 'rsi_diff_recon'):
            # RSI sur l'axe secondaire
            fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['rsi'], mode='lines', name=f'RSI ({RSI_PERIOD})', line=dict(color='purple', width=1)), row=current_row, col=1, secondary_y=True)
            fig.add_hline(y=30, line=dict(color='gray', dash='dash', width=1), row=current_row, col=1, secondary_y=True)
            fig.add_hline(y=70, line=dict(color='gray', dash='dash', width=1), row=current_row, col=1, secondary_y=True)
            
            # Diff et/ou Reconstruction sur l'axe primaire
            if SHOW_DIFF:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['diff_close_sma'], mode='lines', name='Close - SMA', line=dict(color='red', width=1)), row=current_row, col=1, secondary_y=False)
            # La reconstruction sera ajoutée plus tard si activée
            
            current_row += 1

        # Cas 2: RSI seul
        elif any(e for e, t in plot_elements if e == 'rsi_only'):
            fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['rsi'], mode='lines', name=f'RSI ({RSI_PERIOD})', line=dict(color='purple', width=1)), row=current_row, col=1, secondary_y=False)
            fig.add_hline(y=30, line=dict(color='gray', dash='dash', width=1), row=current_row, col=1)
            fig.add_hline(y=70, line=dict(color='gray', dash='dash', width=1), row=current_row, col=1)
            current_row += 1

        # Cas 3: Diff/Reconstruction seuls
        elif any(e for e, t in plot_elements if e == 'diff_recon_only'):
            if SHOW_DIFF:
                fig.add_trace(go.Scatter(x=df_btc.index, y=df_btc['diff_close_sma'], mode='lines', name='Close - SMA', line=dict(color='red', width=1)), row=current_row, col=1, secondary_y=False)
            # La reconstruction sera ajoutée plus tard si activée
            current_row += 1


        # === ANALYSE PAR ONDELETTES MORLET ===
        wavelet_computed = False
        if SHOW_WAVELET:
            print("🌊 Analyse par ondelettes en cours...")
            try:
                # Préparer les données pour l'analyse par ondelettes (utiliser le signal de différence)
                # Utiliser la différence Close - SMA pour analyser les déviations par rapport à la tendance
                signal = df_btc['diff_close_sma'].dropna().values
                time_index = df_btc['diff_close_sma'].dropna().index
                
                # Réduire le signal pour éviter des calculs trop lourds
                max_signal_length = 1024  # Limiter pour performance
                if len(signal) > max_signal_length:
                    step = len(signal) // max_signal_length
                    signal = signal[::step]
                    time_index = time_index[::step]
                
                # Paramètres pour l'analyse par ondelettes optimisés pour basses fréquences
                wavelet = 'morl'  # Ondelette de Morlet (alternative à Daubechies)
                # Étendre la plage d'échelles vers les grandes valeurs pour capturer les basses fréquences
                scales = np.geomspace(1, 256, 80)  # Plus d'échelles avec extension vers 256
                
                # Calculer la transformée en ondelettes continue (CWT)
                coefficients, frequencies = pywt.cwt(signal, scales, wavelet)
                
                # Calculer les pseudo-fréquences pour Morlet
                # Pour Morlet, la fréquence est approximativement 1/scale
                pseudo_frequencies = 1.0 / scales  # Fréquence inversement proportionnelle à l'échelle
                
                # Créer le scalogramme avec normalisation logarithmique pour améliorer le contraste des basses fréquences
                scalogram = np.abs(coefficients)
                
                # Normalisation logarithmique pour accentuer les faibles amplitudes (basses fréquences)
                # Ajouter une petite constante pour éviter log(0)
                epsilon = np.max(scalogram) * 1e-6
                scalogram_log = np.log10(scalogram + epsilon)
                
                # Afficher toutes les fréquences (pas de filtrage)
                scales_filtered = scales
                pseudo_frequencies_filtered = pseudo_frequencies
                scalogram_filtered = scalogram_log
                
                # Ajouter le scalogramme comme heatmap
                fig.add_trace(
                    go.Heatmap(
                        x=time_index,
                        y=scales_filtered,
                        z=scalogram_filtered,
                        colorscale='Plasma',  # Plasma offre un meilleur contraste pour les faibles valeurs
                        name='Ondelettes Morlet',
                        showscale=True,
                        colorbar=dict(
                            title="Log Amplitude", 
                            x=1.02, 
                            len=0.15,  # Réduit la longueur pour mieux s'adapter
                            y=0.35,    # Centré sur le graphique des ondelettes
                            yanchor='middle'
                        )
                    ),
                    row=current_row,
                    col=1
                )
                
                wavelet_computed = True
                current_row += 1
                
            except Exception as e:
                print(f'Erreur analyse ondelettes: {e}')
                import traceback
                traceback.print_exc()
                # Afficher un message sur le graphique en cas d'erreur
                fig.add_annotation(text=f'Erreur ondelettes: {str(e)[:50]}', xref='paper', yref='paper', x=0.5, y=0.4, showarrow=False, row=current_row, col=1)
                current_row += 1

        # === ANALYSE FFT ===
        fft_computed = False
        if SHOW_FFT or SHOW_RECONSTRUCTION:
            print("📈 Analyse FFT en cours...")
            try:
                N = len(difference_values)
                if N >= 4:
                    # Detrend simple: retirer la moyenne
                    vals = difference_values - np.mean(difference_values)
                    # FFT (real-valued optimisée)
                    fft_vals = np.fft.rfft(vals)
                    fft_freqs = np.fft.rfftfreq(N, d=1.0)  # cycles per sample
                    fft_amp = np.abs(fft_vals)

                    # Convertir fréquence -> période (en nombre d'échantillons). On ignore la fréquence 0 (DC) pour la période.
                    nonzero = fft_freqs > 0
                    periods = np.full_like(fft_freqs, np.nan, dtype=float)
                    periods[nonzero] = 1.0 / fft_freqs[nonzero]

                    # Préparer données pour affichage (exclure DC et utiliser les périodes pour une meilleure visualisation)
                    plot_mask = nonzero
                    plot_periods = periods[plot_mask]  # Utiliser les périodes (plus intuitif que les fréquences)
                    plot_amp = fft_amp[plot_mask]

                    # Reconstruction avec filtre des top fréquences dominantes
                    if SHOW_RECONSTRUCTION:
                        try:
                            # Filtre pour garder seulement les top N pics dominants
                            fft_vals_filtered = np.zeros_like(fft_vals)
                            # Garder la composante DC (fréquence 0)
                            fft_vals_filtered[0] = fft_vals[0]
                            
                            # Identifier les top N pics (excluant DC)
                            if len(fft_amp) > 1:
                                k = min(TOP_FREQUENCIES, len(fft_amp) - 1)  # -1 pour exclure DC
                                # Indices des top k amplitudes (excluant DC à l'index 0)
                                top_idx = np.argsort(fft_amp[1:])[-k:] + 1  # +1 pour compenser l'exclusion de DC
                                # Garder seulement ces fréquences
                                fft_vals_filtered[top_idx] = fft_vals[top_idx]
                            
                            # Reconstruction filtrée
                            recon_filtered = np.fft.irfft(fft_vals_filtered, n=N)
                            recon_filtered = recon_filtered + np.mean(difference_values)
                            
                            # Trouver la rangée où tracer la reconstruction
                            recon_row = None
                            for i, (element, _) in enumerate(plot_elements):
                                if element in ['rsi_diff_recon', 'diff_recon_only']:
                                    recon_row = i + 1
                                    break
                            
                            if recon_row:
                                # Tracer la reconstruction filtrée
                                fig.add_trace(
                                    go.Scatter(x=diff_series.index, y=recon_filtered, mode='lines', name=f'Reconstruction top {TOP_FREQUENCIES}', line=dict(color='orange', width=1)),
                                    row=recon_row, col=1
                                )
                            
                        except Exception as e:
                            print(f"Erreur reconstruction: {e}")

                    # Tracer le spectre (Amplitude) si demandé
                    if SHOW_FFT:
                        # Trouver la position de la rangée FFT
                        fft_row = None
                        for i, (element, _) in enumerate(plot_elements):
                            if element == 'fft':
                                fft_row = i + 1
                                break
                        
                        if fft_row:
                            fig.add_trace(
                                go.Scatter(x=plot_periods, y=plot_amp, mode='lines', name='FFT Amplitude', line=dict(color='purple', width=1)),
                                row=fft_row,
                                col=1,
                            )

                            # Annoter les pics dominants
                            try:
                                k = TOP_FREQUENCIES  # Utiliser exactement TOP_FREQUENCIES pour les annotations
                                top_idx = np.argsort(plot_amp)[-k:][::-1]
                                top_periods = plot_periods[top_idx]  # Utiliser les périodes
                                top_amp = plot_amp[top_idx]
                                # Ajouter des marqueurs (sans labels)
                                fig.add_trace(
                                    go.Scatter(x=top_periods, y=top_amp, mode='markers', marker=dict(color='red', size=6), showlegend=False),
                                    row=fft_row,
                                    col=1
                                )
                            except Exception:
                                pass
                    
                    fft_computed = True
                    
                else:
                    if SHOW_FFT:
                        # Trouver la position de la rangée FFT
                        fft_row = None
                        for i, (element, _) in enumerate(plot_elements):
                            if element == 'fft':
                                fft_row = i + 1
                                break
                        
                        if fft_row:
                            # Trop peu de points pour une FFT fiable
                            fig.add_annotation(text='N trop petit pour FFT', xref='paper', yref='paper', x=0.5, y=0.05, showarrow=False, row=fft_row, col=1)
                    
            except Exception as e:
                print('Erreur FFT:', e)
                if SHOW_FFT:
                    # Trouver la position de la rangée FFT
                    fft_row = None
                    for i, (element, _) in enumerate(plot_elements):
                        if element == 'fft':
                            fft_row = i + 1
                            break
                    
                    if fft_row:
                        fig.add_annotation(text=f'Erreur FFT: {str(e)[:30]}', xref='paper', yref='paper', x=0.5, y=0.05, showarrow=False, row=fft_row, col=1)

        # Ajouter une ligne horizontale fine et pointillée (y=0) sur la rangée des différences si elle existe
        if SHOW_DIFF or SHOW_RECONSTRUCTION:
            diff_recon_row = None
            for i, (element, _) in enumerate(plot_elements):
                if element in ['rsi_diff_recon', 'diff_recon_only']:
                    diff_recon_row = i + 1
                    break
            
            if diff_recon_row:
                try:
                    fig.add_hline(y=0, line=dict(color='gray', dash='dot', width=1), row=diff_recon_row, col=1)
                except Exception:
                    try:
                        fig.add_shape(
                            type='line',
                            x0=df_btc.index.min(), x1=df_btc.index.max(),
                            y0=0, y1=0,
                            xref='x', yref='y',
                            line=dict(color='gray', dash='dot', width=1),
                            row=diff_recon_row, col=1,
                        )
                    except Exception:
                        pass

        # Layout improvements
        fig.update_layout(
            height=400 + num_plots * 300,  # Hauteur adaptative selon le nombre de plots
            showlegend=False,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            hovermode='x unified',
            margin=dict(l=60, r=60, t=80, b=120),
        )

        # Configuration des axes Y selon les éléments affichés
        row_idx = 1
        for element, _ in plot_elements:
            if element == 'price_volume':
                fig.update_yaxes(title_text='Price', row=row_idx, col=1, secondary_y=False)
                fig.update_yaxes(title_text='Volume', row=row_idx, col=1, secondary_y=True)
            elif element == 'rsi_diff_recon':
                fig.update_yaxes(title_text='Diff', row=row_idx, col=1, secondary_y=False)
                fig.update_yaxes(title_text='RSI', row=row_idx, col=1, secondary_y=True, range=[0, 100])
            elif element == 'rsi_only':
                fig.update_yaxes(title_text='RSI', row=row_idx, col=1, range=[0, 100])
            elif element == 'diff_recon_only':
                fig.update_yaxes(title_text='Diff', row=row_idx, col=1)
            elif element == 'wavelet':
                fig.update_yaxes(title_text='Échelles', row=row_idx, col=1)
            elif element == 'fft':
                fig.update_yaxes(title_text='Amplitude', row=row_idx, col=1)
            row_idx += 1

        # Configuration des axes X pour la synchronisation
        date_xargs = dict(type='date', tickformat="%Y-%m-%d\n%H:%M", tickangle=-45, tickfont=dict(size=10), nticks=8, ticks='outside', showgrid=False, showticklabels=True, title_standoff=20)
        
        # Identifier les rangées qui ne sont pas FFT (pour synchronisation)
        non_fft_rows = []
        for i, (element, _) in enumerate(plot_elements):
            if element != 'fft':
                non_fft_rows.append(i + 1)
        
        try:
            x0 = df_btc.index.min()
            x1 = df_btc.index.max()
            
            # Synchroniser les axes X des rangées non-FFT
            if non_fft_rows:
                for i, row in enumerate(non_fft_rows):
                    if i == 0:
                        # Première rangée non-FFT comme référence
                        fig.update_xaxes(range=[x0, x1], title_text='', matches='x4', **date_xargs, row=row, col=1)
                    else:
                        # Les autres se synchronisent avec la première
                        fig.update_xaxes(range=[x0, x1], title_text='', matches='x4', **date_xargs, row=row, col=1)
            
            # Configuration spéciale pour FFT si présent
            for i, (element, _) in enumerate(plot_elements):
                if element == 'fft':
                    try:
                        fig.update_xaxes(title_text='Période (heures)', row=i+1, col=1, type='log')
                    except Exception:
                        pass
                    
        except Exception:
            # Fallback en cas d'erreur
            for i in range(1, num_plots + 1):
                fig.update_xaxes(title_text='Time', **date_xargs, row=i, col=1)

        # Affichage simple
        print("📊 Affichage du graphique...")
        try:
            fig.show(renderer='vscode')
        except Exception as e:
            print("Erreur: impossible d'afficher la figure avec le renderer 'vscode'.")
            print("Détail: ", str(e))

# Connecter les callbacks aux boutons
download_button.on_click(on_download_click)
visualize_button.on_click(on_visualize_click)

# Fonction pour mettre à jour l'affichage de la configuration
def update_config_display():
    with output_area:
        clear_output(wait=False)
        #print("Configuration actuelle:")
        #print(f"- Symbole: {symbol_dropdown.value}")
        #print(f"- Durée d'analyse: {duree_slider.value} jours")
        #print(f"- Intervalle: {interval_dropdown.value}")
        #print(f"- Date de fin: {end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else 'Non définie'}")
        #print("\nCliquez sur 'Télécharger les données' pour commencer.")

# Callback pour mettre à jour l'affichage quand les sliders changent
def on_slider_change(change):
    #update_config_display()
    # Mettre à jour les variables globales en temps réel
    global DUREE_JOURS, INTERVAL, SYMBOL, END_DATE, SMA_PERIOD, EMA_PERIOD, EMA2_PERIOD, HMA_PERIOD, ZLEMA_PERIOD, TEMA_PERIOD, TOP_FREQUENCIES, RSI_PERIOD
    global SHOW_PRICE, SHOW_SMA, SHOW_EMA, SHOW_EMA2, SHOW_HMA, SHOW_ZLEMA, SHOW_TEMA, SHOW_VOLUME, SHOW_DIFF, SHOW_RSI, SHOW_WAVELET, SHOW_FFT, SHOW_RECONSTRUCTION
    DUREE_JOURS = duree_slider.value
    INTERVAL = interval_dropdown.value
    SYMBOL = symbol_dropdown.value
    END_DATE = end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else datetime.now().strftime('%Y-%m-%d')
    SMA_PERIOD = sma_period_slider.value
    EMA_PERIOD = ema_period_slider.value
    EMA2_PERIOD = ema2_period_slider.value
    HMA_PERIOD = hma_period_slider.value
    ZLEMA_PERIOD = zlema_period_slider.value
    TEMA_PERIOD = tema_period_slider.value
    TOP_FREQUENCIES = top_freq_slider.value
    RSI_PERIOD = rsi_period_slider.value
    SHOW_PRICE = show_price_checkbox.value
    SHOW_SMA = show_sma_checkbox.value
    SHOW_EMA = show_ema_checkbox.value
    SHOW_EMA2 = show_ema2_checkbox.value
    SHOW_HMA = show_hma_checkbox.value
    SHOW_ZLEMA = show_zlema_checkbox.value
    SHOW_TEMA = show_tema_checkbox.value
    SHOW_VOLUME = show_volume_checkbox.value
    SHOW_DIFF = show_diff_checkbox.value
    SHOW_RSI = show_rsi_checkbox.value
    SHOW_WAVELET = show_wavelet_checkbox.value
    SHOW_FFT = show_fft_checkbox.value
    SHOW_RECONSTRUCTION = show_reconstruction_checkbox.value
    
    # Si les données sont déjà chargées, mettre à jour automatiquement la visualisation
    #if len(df_btc) > 0 and (change.owner == sma_period_slider or change.owner == top_freq_slider):
    #    with output_area:
    #        clear_output(wait=True)
    #        print(f"=== MISE À JOUR DE LA VISUALISATION ===")
    #        print(f"Nouvelle période SMA: {SMA_PERIOD}, FFT Pics: {TOP_FREQUENCIES}")
    #        print()
    #        create_visualization()

duree_slider.observe(on_slider_change, names='value')
interval_dropdown.observe(on_slider_change, names='value')
symbol_dropdown.observe(on_slider_change, names='value')
end_date_picker.observe(on_slider_change, names='value')
sma_period_slider.observe(on_slider_change, names='value')
ema_period_slider.observe(on_slider_change, names='value')
ema2_period_slider.observe(on_slider_change, names='value')
hma_period_slider.observe(on_slider_change, names='value')
zlema_period_slider.observe(on_slider_change, names='value')
tema_period_slider.observe(on_slider_change, names='value')
top_freq_slider.observe(on_slider_change, names='value')
rsi_period_slider.observe(on_slider_change, names='value')

# Ajouter les observers pour les checkboxes
show_price_checkbox.observe(on_slider_change, names='value')
show_sma_checkbox.observe(on_slider_change, names='value')
show_ema_checkbox.observe(on_slider_change, names='value')
show_ema2_checkbox.observe(on_slider_change, names='value')
show_hma_checkbox.observe(on_slider_change, names='value')
show_zlema_checkbox.observe(on_slider_change, names='value')
show_tema_checkbox.observe(on_slider_change, names='value')
show_volume_checkbox.observe(on_slider_change, names='value')
show_diff_checkbox.observe(on_slider_change, names='value')
show_rsi_checkbox.observe(on_slider_change, names='value')
show_wavelet_checkbox.observe(on_slider_change, names='value')
show_fft_checkbox.observe(on_slider_change, names='value')
show_reconstruction_checkbox.observe(on_slider_change, names='value')

# Créer des labels séparés pour un meilleur alignement
symbol_label = Label(value='Symbole:', layout={'width': '120px'})
duree_label = Label(value='Durée (jours):', layout={'width': '120px'})
interval_label = Label(value='Intervalle:', layout={'width': '120px'})
end_date_label = Label(value='Date de fin:', layout={'width': '120px'})

# Labels pour les checkboxes d'affichage (groupés avec leurs paramètres)
price_checkbox_label = Label(value='Prix', layout={'width': '40px'})
volume_checkbox_label = Label(value='Volume', layout={'width': '50px'})
diff_checkbox_label = Label(value='Diff', layout={'width': '35px'})
wavelet_checkbox_label = Label(value='Ondelettes', layout={'width': '80px'})
reconstruction_checkbox_label = Label(value='Reconstruction', layout={'width': '100px'})

# Supprimer les descriptions des widgets pour éviter la duplication
symbol_dropdown.description = ''
duree_slider.description = ''
interval_dropdown.description = ''
end_date_picker.description = ''
sma_period_slider.description = ''
ema_period_slider.description = ''
ema2_period_slider.description = ''
hma_period_slider.description = ''
zlema_period_slider.description = ''
tema_period_slider.description = ''
top_freq_slider.description = ''
rsi_period_slider.description = ''

# Créer une seule "card" pour tous les contrôles
main_card = VBox([
    # Première ligne: Symbole
    HBox([symbol_label, symbol_dropdown], layout={'justify_content': 'flex-start'}),
    # Deuxième ligne: Durée
    HBox([duree_label, duree_slider], layout={'justify_content': 'flex-start'}),
    # Troisième ligne: Intervalle
    HBox([interval_label, interval_dropdown], layout={'justify_content': 'flex-start'}),
    # Quatrième ligne: Date de fin
    HBox([end_date_label, end_date_picker], layout={'justify_content': 'flex-start'}),
    # Prix
    HBox([show_price_checkbox, price_checkbox_label], layout={'justify_content': 'flex-start'}),
    # SMA avec checkbox et slider sur la même ligne
    HBox([show_sma_checkbox, Label(value='SMA', layout={'width': '40px'}), sma_period_slider], layout={'justify_content': 'flex-start'}),
    # EMA avec checkbox et slider sur la même ligne
    HBox([show_ema_checkbox, Label(value='EMA', layout={'width': '40px'}), ema_period_slider], layout={'justify_content': 'flex-start'}),
    # EMA 2 avec checkbox et slider sur la même ligne
    HBox([show_ema2_checkbox, Label(value='EMA 2', layout={'width': '45px'}), ema2_period_slider], layout={'justify_content': 'flex-start'}),
    # HMA avec checkbox et slider sur la même ligne
    HBox([show_hma_checkbox, Label(value='HMA', layout={'width': '45px'}), hma_period_slider], layout={'justify_content': 'flex-start'}),
    # ZLEMA avec checkbox et slider sur la même ligne
    HBox([show_zlema_checkbox, Label(value='ZLEMA', layout={'width': '45px'}), zlema_period_slider], layout={'justify_content': 'flex-start'}),
    # TEMA avec checkbox et slider sur la même ligne
    HBox([show_tema_checkbox, Label(value='TEMA', layout={'width': '45px'}), tema_period_slider], layout={'justify_content': 'flex-start'}),
    # Volume
    HBox([show_volume_checkbox, volume_checkbox_label], layout={'justify_content': 'flex-start'}),
    # Diff
    HBox([show_diff_checkbox, diff_checkbox_label], layout={'justify_content': 'flex-start'}),
    # RSI avec checkbox et slider sur la même ligne
    HBox([show_rsi_checkbox, Label(value='RSI', layout={'width': '40px'}), rsi_period_slider], layout={'justify_content': 'flex-start'}),
    # Ondelettes
    HBox([show_wavelet_checkbox, wavelet_checkbox_label], layout={'justify_content': 'flex-start'}),
    # FFT avec checkbox et slider sur la même ligne
    HBox([show_fft_checkbox, Label(value='FFT', layout={'width': '40px'}), top_freq_slider], layout={'justify_content': 'flex-start'}),
    # Reconstruction
    HBox([show_reconstruction_checkbox, reconstruction_checkbox_label], layout={'justify_content': 'flex-start'}),
    # Boutons (séparés par un peu d'espace)
    HBox([download_button, visualize_button], layout={'justify_content': 'flex-start', 'margin': '15px 0px 0px 0px'})
], layout={
    'border': '2px solid #e0e0e0',
    'border_radius': '8px',
    'padding': '15px',
    'margin': '10px 0px',
    'background_color': '#f8f9fa',
    'width': '100%'
})

display(main_card)
display(output_area)

# Affichage initial de la configuration
update_config_display()

warnings.filterwarnings('ignore')

def download_binance_vision_data(symbol=SYMBOL, interval=INTERVAL, days_back=DUREE_JOURS, end_date_str=None):

    # Résolution de la date de fin
    if end_date_str is None:
        try:
            end_date_str = END_DATE  # variable globale définie dans la cellule de config
        except NameError:
            end_date_str = None

    if end_date_str:
        try:
            end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
        except Exception:
            print(f"Format END_DATE invalide: {end_date_str}. Utilisation de datetime.now().")
            end_date = datetime.now()
    else:
        end_date = datetime.now()

    # Calculer la date de début
    start_date = end_date - timedelta(days=days_back)

    # Construire la liste des dates à télécharger
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    # Création des widgets de progression
    progress_bar = FloatProgress(
        value=0,
        min=0,
        max=len(dates),
        bar_style='info',
        style={'bar_color': '#00c851'},
        orientation='horizontal'
    )
    
    # status_label = HTML(value=f"<b>Préparation du téléchargement de {len(dates)} fichiers...</b>")
    
    # Conteneur pour afficher la barre de progression
    progress_widget = VBox([progress_bar])
    display(progress_widget)

    all_data = []
    success_count = 0
    total_periods = 0

    from concurrent.futures import ThreadPoolExecutor, as_completed

    def fetch_for_date(date_str):
        """Télécharge et retourne un tuple (date_str, df_day or None, error_message or None, periods_count)"""
        url = f"https://data.binance.vision/data/spot/daily/klines/{symbol}/{interval}/{symbol}-{interval}-{date_str}.zip"
        try:
            # Petite trace locale (renvoyée pour agrégation)
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                with zipfile.ZipFile(BytesIO(resp.content)) as zip_file:
                    csv_filename = f"{symbol}-{interval}-{date_str}.csv"
                    if csv_filename in zip_file.namelist():
                        csv_content = zip_file.read(csv_filename)
                        df_day = pd.read_csv(BytesIO(csv_content), header=None, names=[
                            'timestamp', 'open', 'high', 'low', 'close', 'volume',
                            'close_time', 'quote_asset_volume', 'number_of_trades',
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                        ])
                        return (date_str, df_day, None, len(df_day))
                    else:
                        return (date_str, None, 'CSV not found in ZIP', 0)
            else:
                return (date_str, None, f'HTTP {resp.status_code}', 0)
        except requests.exceptions.RequestException:
            return (date_str, None, 'Network error', 0)
        except zipfile.BadZipFile:
            return (date_str, None, 'Bad ZIP', 0)
        except Exception as e:
            return (date_str, None, str(e)[:200], 0)

    # Paramètres de parallélisme: limiter le nombre de threads raisonnablement
    max_workers = min(12, max(4, len(dates)))
    completed_count = 0

    with ThreadPoolExecutor(max_workers=max_workers) as exe:
        future_to_date = {exe.submit(fetch_for_date, d): d for d in dates}
        for fut in as_completed(future_to_date):
            date_str = future_to_date[fut]
            try:
                d, df_day, err, periods = fut.result()
                completed_count += 1
                
                # Mise à jour de la barre de progression
                progress_bar.value = completed_count
                
                if df_day is not None:
                    # --- Normalisation des timestamps: uniformiser en microsecondes (us) ---
                    try:
                        # S'assurer que la colonne 'timestamp' existe et est numérique
                        if 'timestamp' not in df_day.columns:
                            # si les colonnes sont indexées numériquement, la première colonne est le timestamp
                            df_day.rename(columns={0: 'timestamp'}, inplace=True)
                        df_day['timestamp'] = pd.to_numeric(df_day['timestamp'], errors='coerce')
                    except Exception:
                        pass

                    try:
                        # Détecter les timestamps inférieurs au seuil (2025-01-01)
                        threshold_dt = datetime(2025, 1, 1)
                        # Seuil en millisecondes correspondant à 2025-01-01
                        threshold_ms = int(threshold_dt.timestamp() * 1000)

                        # Heuristique: si la majorité des timestamps sont < threshold_ms*10, ils sont probablement en ms
                        max_ts = pd.to_numeric(df_day['timestamp'], errors='coerce').max()
                        if pd.notna(max_ts):
                            # Si le maximum observé est inférieur à threshold_ms * 10, on suppose que les valeurs sont en ms
                            if max_ts < threshold_ms * 10:
                                mask_ms = pd.to_numeric(df_day.loc[:, 'timestamp'], errors='coerce') < (threshold_ms * 10)
                                if mask_ms.any():
                                    df_day.loc[mask_ms, 'timestamp'] = pd.to_numeric(df_day.loc[mask_ms, 'timestamp'], errors='coerce') * 1000
                    except Exception as e:
                        print(f"⚠ Erreur lors de la normalisation des timestamps: {e}")

                    all_data.append(df_day)
                    success_count += 1
                    total_periods += periods
                    
            except Exception as e:
                completed_count += 1
                progress_bar.value = completed_count
               
    # Finalisation de la barre de progression
    progress_bar.bar_style = 'success' if success_count > 0 else 'danger'
   
    # Petit délai global pour rester poli si nécessaire
    time.sleep(0.1)

    if all_data:
        # Combiner toutes les données
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df.to_dict('records')
    else:
        print("Aucune donnée récupérée via Binance Vision")
        return []

def create_dataframe(data, symbol_name):
    if not data:
        print(f"Aucune donnée disponible pour {symbol_name}")
        return pd.DataFrame()
        
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    # Gestion des timestamps - Binance Vision utilise différents formats
    timestamp_converted = False
    
    try:
        # Essai avec microsecondes (format attendu après normalisation)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
        timestamp_converted = True
        # print("✓ Timestamps convertis depuis microsecondes")
    except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
        try:
            # Essai avec millisecondes (fallback)
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            timestamp_converted = True
        except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
            try:
                # Essai avec secondes
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                timestamp_converted = True
            except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                try:
                    # Si les valeurs sont trop grandes, diviser par 1000000 (microsecondes vers secondes)
                    df['timestamp'] = pd.to_datetime(df['timestamp'] / 1000000, unit='s')
                    timestamp_converted = True
                except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                    print("✗ Impossible de convertir les timestamps")

    if not timestamp_converted:
        print(f"✗ Erreur: Impossible de traiter les timestamps pour {symbol_name}")
        return pd.DataFrame()
    
    # Vérifier la plausibilité des dates
    min_date = df['timestamp'].min()
    max_date = df['timestamp'].max()
    
    if min_date.year < 2009 or max_date.year > 2030:
        print(f"⚠ Dates suspectes détectées: {min_date} à {max_date}")
    
    df.set_index('timestamp', inplace=True)
    
    # Supprimer les doublons et trier
    df = df[~df.index.duplicated(keep='first')].sort_index()

    
    return df


Output()